# Filtering and merging the dataframes
The dataset used has almost half a million rows, most of which are filled with missing values. So, in order to get to web scraping the images of the objects within the project's timeframe, I had to reassess my options and I decided to only keep artworks with gallery numbers (because that would mean that those are displayed), only keep the Fifth Avenue Museum (discarding the Met Cloisters) and keep only the artworks with images, which lead to having only around 33.000 rows (an easier number to manage and web scrape).

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

pd.set_option('display.max_columns', None)

artworks_df = pd.read_csv('../data/clean/artworks_df.csv', low_memory=False)
museum_df = pd.read_csv('../data/clean/museum_df.csv', low_memory=False)
display(artworks_df.shape)
display(museum_df.shape)

In [ ]:
images_and_tags = pd.read_csv('../data/clean/images_and_tags.csv', low_memory=False)
images_and_tags = images_and_tags.dropna(subset=['image_url'])
images_and_tags = images_and_tags.drop(columns = ['Unnamed: 0'])
images_and_tags = images_and_tags[images_and_tags['object_id'].isin(artworks_df['object_id'])]
images_and_tags.reset_index(drop=True, inplace=True)

# Verify the result
print(f"Number of rows in images_and_tags after filtering: {len(images_and_tags)}")
print(f"Number of rows in artworks_df: {len(artworks_df)}")

In [ ]:
artworks_df = artworks_df[artworks_df['object_id'].isin(images_and_tags['object_id'])]
artworks_df.reset_index(drop=True, inplace=True)
print(f"Number of rows in artworks_df after filtering: {len(artworks_df)}")

In [ ]:
# Filter artworks_df to match the object_ids present in museum_df
artworks_df = artworks_df[artworks_df['object_id'].isin(museum_df['object_id'])]

# Optional: Reset the index of artworks_df after filtering
artworks_df.reset_index(drop=True, inplace=True)

# Verify the result
print(f"Number of rows in artworks_df after filtering: {len(artworks_df)}")
print(f"Number of rows in museum_df: {len(museum_df)}")

# Filter museum_df to match the object_ids present in artworks_df
museum_df = museum_df[museum_df['object_id'].isin(artworks_df['object_id'])]

# Optional: Reset the index of museum_df after filtering
museum_df.reset_index(drop=True, inplace=True)

# Verify the result
print(f"Number of rows in museum_df after filtering: {len(museum_df)}")
print(f"Number of rows in artworks_df: {len(artworks_df)}")

In [ ]:
museum_df = museum_df.drop_duplicates(subset='object_id')
# Recheck the number of rows
print(f"Number of rows in museum_df after re-filtering: {len(museum_df)}")

In [ ]:
geo_df = pd.read_csv('../data/clean/geo_df.csv', low_memory=False)
geo_df = geo_df[geo_df['object_id'].isin(artworks_df['object_id'])]
geo_df.reset_index(drop=True, inplace=True)

print(f"Number of rows in geo_df after filtering: {len(geo_df)}")
print(f"Number of rows in artworks_df: {len(artworks_df)}")

In [ ]:
linking_table = pd.read_csv('../data/clean/linking_table.csv', low_memory=False)
linking_table = linking_table[linking_table['object_id'].isin(artworks_df['object_id'])]
linking_table.reset_index(drop=True, inplace=True)
artworks_df = artworks_df[artworks_df['object_id'].isin(linking_table['object_id'])]
artworks_df.reset_index(drop=True, inplace=True)
# Verify the result
print(f"Number of rows in linking_table after filtering: {len(linking_table)}")
print(f"Number of rows in artworks_df: {len(artworks_df)}")

In [ ]:
def preprocess_tags(df, tag_column='tags'):
    """
    Splits tags in the specified column into a list.
    Handles edge cases where the column might have NaN or unexpected formats.
    """
    def split_tags(tag_value):
        # Check if the value is a string and split on the exact delimiter
        if isinstance(tag_value, str):
            return [tag.strip() for tag in tag_value.split('|')]  # Adjusted for no spaces around "|"
        return []  # Return an empty list for non-string or missing values

    # Apply the splitting function
    df[tag_column] = df[tag_column].apply(split_tags)
    return df

images_and_tags = preprocess_tags(images_and_tags)
images_and_tags

In [ ]:
artists_df = pd.read_csv('../data/clean/artists_df.csv', low_memory=False)
print(f"Number of rows in artists_df before filtering: {len(artists_df)}")
artists_df = artists_df[artists_df['artist_id'].isin(linking_table['artist_id'])]
artists_df.reset_index(drop=True, inplace=True)

# Verify the result
print(f"Number of rows in artists_df after filtering: {len(artists_df)}")

In [ ]:
linking_table['artist_id'].nunique()

In [ ]:
# images_and_tags.to_csv('../art-sense/data/clean/images_and_tags.csv')
linking_table = linking_table.drop(columns = 'Unnamed: 0')
display(linking_table)

In [ ]:
def preprocess_tags_multiple_columns_preserve_nan(df, columns):
    """
    Splits tags in the specified columns into lists while preserving NaN values.
    """
    def split_tags(tag_value):
        # Only split if the value is a string, otherwise return an empty list
        if isinstance(tag_value, str):
            # Split by '|' and remove any extra spaces around each tag
            return [tag.strip() for tag in tag_value.split('|')]  
        else:
            return []  # Return empty list if the value is NaN or not a string

    # Apply the split_tags function across the specified columns
    for column in columns:
        if column in df.columns:
            df[column] = df[column].apply(split_tags)
        else:
            print(f"Warning: Column '{column}' not found in DataFrame.")
    
    return df

geo_columns = ['geography_type', 'city', 'state', 'county', 'country',
               'region', 'subregion', 'locale', 'locus', 'excavation', 'river']

# Example of how to apply it to your geo_df
geo_df = preprocess_tags_multiple_columns_preserve_nan(geo_df, geo_columns)
geo_df

In [ ]:
artists_df['artist_display_name'].fillna('Unknown', inplace=True)
artists_df.head(50)

In [20]:
linking_table.to_csv('../data/clean/linking_table.csv', index= False)
artists_df.to_csv('../data/clean/artists.csv', index= False)
geo_df.to_csv('../data/clean/geo.csv', index= False)
museum_df.to_csv('../data/clean/museum.csv', index= False)
artworks_df.to_csv('../data/clean/artworks.csv', index= False)
images_and_tags.to_csv('../data/clean/images_and_tags.csv', index= False)

In [5]:
# Merge datasets
merged_df = (artworks_df
             .merge(linking_table_df, on='object_id', how='left')
             .merge(artists_df, on='artist_id', how='left')
             .merge(geo_df, on='object_id', how='left')
             .merge(images_and_tags_df, on='object_id', how='left')
             .merge(museum_df, on='object_id', how='left'))

In [ ]:
merged_df = merged_df.drop(columns = ['link_resource_x'])
merged_df.rename(columns={'link_resource_y': 'link_resource'}, inplace=True)
merged_df.columns
merged_df.to_csv('../data/clean/full_dataset.csv', index= False)